In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\camilo\anaconda3\lib\site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

koi_disposition  koi_fpflag_nt  koi_fpflag_ss  koi_fpflag_co  koi_fpflag_ec  \
0       CONFIRMED              0              0              0              0   
1  FALSE POSITIVE              0              1              0              0   
2  FALSE POSITIVE              0              1              0              0   
3       CONFIRMED              0              0              0              0   
4       CONFIRMED              0              0              0              0   

   koi_period  koi_period_err1  koi_period_err2  koi_time0bk  \
0   54.418383     2.479000e-04    -2.479000e-04   162.513840   
1   19.899140     1.490000e-05    -1.490000e-05   175.850252   
2    1.736952     2.630000e-07    -2.630000e-07   170.307565   
3    2.525592     3.760000e-06    -3.760000e-06   171.595550   
4    4.134435     1.050000e-05    -1.050000e-05   172.979370   

   koi_time0bk_err1  ...  koi_steff_err2  koi_slogg  koi_slogg_err1  \
0          0.003520  ...             -81      4.467           0.064   
1          0.000581  ...            -176      4.544           0.044   
2          0.000115  ...            -174      4.564           0.053   
3          0.001130  ...            -211      4.438           0.070   
4          0.001900  ...            -232      4.486           0.054   

   koi_slogg_err2  koi_srad  koi_srad_err1  koi_srad_err2         ra  \
0          -0.096     0.927          0.105         -0.061  291.93423   
1          -0.176     0.868          0.233         -0.078  297.00482   
2          -0.168     0.791          0.201         -0.067  285.53461   
3          -0.210     1.046          0.334         -0.133  288.75488   
4          -0.229     0.972          0.315         -0.105  296.28613   

         dec  koi_kepmag  
0  48.141651      15.347  
1  48.134129      15.436  
2  48.285210      15.597  
3  48.226200      15.509  
4  48.224670      15.714  

[5 rows x 41 columns]

# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_time0bk', 'koi_impact',
       'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_slogg',
       'koi_srad', 'ra', 'dec', 'koi_kepmag']]

In [6]:
y = df[["koi_disposition"]]

In [7]:
y = y.values.reshape(-1,)
y.shape

(6991,)

# Create a Train Test Split

Use `koi_disposition` for the y values

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(selected_features, y, test_size=0.20)

In [9]:
X_train.head()

koi_fpflag_nt  koi_fpflag_ss  koi_fpflag_co  koi_fpflag_ec  koi_period  \
3142              0              0              0              0   44.985188   
5534              0              1              0              0   12.180721   
5738              0              1              0              0   15.446060   
2223              0              0              0              0    6.232123   
4833              0              0              0              0  651.358046   

      koi_time0bk  koi_impact  koi_duration  koi_depth  koi_prad  koi_teq  \
3142   150.570300      0.6640       4.16400      617.8      2.09      433   
5534   139.346004      0.4790       4.15602   414950.0     63.25      742   
5738   145.534127      0.3000       4.59630   330910.0     52.93      701   
2223   135.236900      0.3200       2.85000       31.7      0.61     1021   
4833   213.181620      0.0946       4.38400      570.8     14.58      411   

      koi_insol  koi_model_snr  koi_tce_plnt_num  koi_steff  koi_slogg  \
3142       8.34           12.7                 2       5295      4.548   
5534      71.88         1809.8                 1       5750      4.538   
5738      57.04         2025.2                 1       5838      4.529   
2223     256.72            8.5                 3       5843      4.421   
4833       6.75           11.0                 1       5122      3.172   

      koi_srad         ra        dec  koi_kepmag  
3142     0.800  295.90503  45.405430      15.850  
5534     0.882  284.31717  47.678638      15.386  
5738     0.897  296.43854  43.965961      15.488  
2223     1.084  287.97885  42.679359      12.857  
4833     6.261  291.26584  42.739491      12.888

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [10]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Train the Model



In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()

model.add(Dense(units=100, activation='relu', input_dim=20))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [13]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [14]:
model.fit(X_train_scaled,
         y_train_categorical,
         epochs=100,
         shuffle=True,
         verbose=2)

Train on 5592 samples
Epoch 1/100
5592/5592 - 0s - loss: 0.5254 - accuracy: 0.7235
Epoch 2/100
5592/5592 - 0s - loss: 0.3807 - accuracy: 0.7897
Epoch 3/100
5592/5592 - 0s - loss: 0.3730 - accuracy: 0.7940
Epoch 4/100
5592/5592 - 0s - loss: 0.3657 - accuracy: 0.7954
Epoch 5/100
5592/5592 - 0s - loss: 0.3635 - accuracy: 0.7994
Epoch 6/100
5592/5592 - 0s - loss: 0.3599 - accuracy: 0.8117
Epoch 7/100
5592/5592 - 0s - loss: 0.3564 - accuracy: 0.8135
Epoch 8/100
5592/5592 - 0s - loss: 0.3539 - accuracy: 0.8149
Epoch 9/100
5592/5592 - 0s - loss: 0.3515 - accuracy: 0.8112
Epoch 10/100
5592/5592 - 0s - loss: 0.3499 - accuracy: 0.8144
Epoch 11/100
5592/5592 - 0s - loss: 0.3506 - accuracy: 0.8133
Epoch 12/100
5592/5592 - 0s - loss: 0.3457 - accuracy: 0.8180
Epoch 13/100
5592/5592 - 0s - loss: 0.3464 - accuracy: 0.8224
Epoch 14/100
5592/5592 - 0s - loss: 0.3462 - accuracy: 0.8185
Epoch 15/100
5592/5592 - 0s - loss: 0.3406 - accuracy: 0.8258
Epoch 16/100
5592/5592 - 0s - loss: 0.3384 - accuracy: 0.

In [15]:
print(f"Training Data Score: {model.evaluate(X_train_scaled, y_train_categorical)}")
print(f"Testing Data Score: {model.evaluate(X_test_scaled, y_test_categorical)}")

5592/5592 [==============================] - 0s 36us/sample - loss: 0.2248 - accuracy: 0.8902
Training Data Score: [0.22476367592726312, 0.89020026]
1399/1399 [==============================] - 0s 30us/sample - loss: 0.3428 - accuracy: 0.8620
Testing Data Score: [0.3427723944826242, 0.86204433]


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [23]:
# Create the GridSearchCV model

In [ ]:
# Train the model with GridSearch

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)